In [18]:
!python save_cpics_pngs_to_lmdb.py --dataset_path="/home/nick/Downloads/113201/FlowCamNet/imgs" --lmdb_dir_name="/home/nick/Documents/ws24/lmdb/bigger/" --min_size=128 --dataset_name="ZooScan"

wtf
TOTAL #images 301247 FROM /home/nick/Downloads/113201/FlowCamNet/imgs
100%|█████████████████████████████████| 301247/301247 [02:05<00:00, 2391.53it/s]
Finished importing from /home/nick/Downloads/113201/FlowCamNet/imgs and subdirectories, saved at: /home/nick/Documents/ws24/lmdb/bigger/ZooScan_imgs
Finished importing from /home/nick/Downloads/113201/FlowCamNet/imgs and subdirectories, saved at: /home/nick/Documents/ws24/lmdb/bigger/ZooScan_imgs


For ZooScan 

60115 imgs remain if only one of the dims has to be > 128, 
13k imgs if both

### Verify correct labels were assigned to correct img

In [20]:
import os
import lmdb
import imageio.v3 as iio
import numpy as np

# Set paths to your LMDB directories
LMDB_IMGS_PATH = "/home/nick/Documents/ws24/lmdb/bigger/ZooScan_imgs" #"/home/nick/Documents/ws24/out/TEST_imgs" #
LMDB_LABELS_PATH = "/home/nick/Documents/ws24/lmdb/bigger/ZooScan_labels" #"/home/nick/Documents/ws24/out/TEST_labels"
DEBUG_OUTPUT_DIR = "./debug_output"


os.makedirs(DEBUG_OUTPUT_DIR, exist_ok=True)
def count_entries(lmdb_path):
    env = lmdb.open(lmdb_path, readonly=True)
    
    with env.begin() as txn:
        cursor = txn.cursor()
        count = 0
        for key, _ in cursor:
            count += 1
        
    env.close()
    
    return count

def load_and_verify_lmdb(lmdb_imgs_path, lmdb_labels_path, debug_output_dir, max_images=10):
    num_images = count_entries(lmdb_imgs_path)
    num_labels = count_entries(lmdb_labels_path)
    
    print(f"Number of images in LMDB: {num_images}")
    print(f"Number of labels in LMDB: {num_labels}")

    env_imgs = lmdb.open(lmdb_imgs_path, readonly=True)
    env_labels = lmdb.open(lmdb_labels_path, readonly=True)

    with env_imgs.begin() as txn_imgs, env_labels.begin() as txn_labels:
        cursor_imgs = txn_imgs.cursor()
        cursor_labels = txn_labels.cursor()        
        count = 0
        for (img_key, img_value), (label_key, label_value) in zip(cursor_imgs, cursor_labels):
            img_decoded = iio.imread(img_value)
            
            label_decoded = label_value.decode("utf-8")
            
            # Save the image for verification
            output_path = os.path.join(debug_output_dir, f"debug_{label_decoded}_{count}.png")
            iio.imwrite(output_path, img_decoded)
            
            print(f"Verified Image Key: {img_key.decode('utf-8')}, Label: {label_decoded}, Saved to: {output_path}")
            
            count += 1
            if count >= max_images:  # dont want to save all images
                break

    env_imgs.close()
    env_labels.close()
    print(f"Verification complete. Images saved to {debug_output_dir}")

load_and_verify_lmdb(LMDB_IMGS_PATH, LMDB_LABELS_PATH, DEBUG_OUTPUT_DIR)


Number of images in LMDB: 60115
Number of labels in LMDB: 60115
Verified Image Key: Acantharia_164727498.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_0.png
Verified Image Key: Acantharia_164727514.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_1.png
Verified Image Key: Acantharia_164727544.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_2.png
Verified Image Key: Acantharia_164728219.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_3.png
Verified Image Key: Acantharia_164901027.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_4.png
Verified Image Key: Acantharia_164966858.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_5.png
Verified Image Key: Acantharia_164969210.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_6.png
Verified Image Key: Acantharia_164970897.png, Label: Acantharia, Saved to: ./debug_output/debug_Acantharia_7.png
Verified Image Key: Acantharia_1

In [ ]:
import os
import lmdb
import imageio.v3 as iio
import numpy as np
from tqdm import tqdm
import psutil
from sklearn.model_selection import train_test_split

def get_available_memory():
    """Get available memory and return a safe allocation size for LMDB."""
    available_memory = psutil.virtual_memory().available
    print(f"Available memory: {available_memory / 1024 / 1024} MB")
    return int(available_memory * 0.75) 


MAP_SIZE_IMG = get_available_memory()
MAP_SIZE_META = int(get_available_memory()*0.1)

def load_lmdb_data(lmdb_path):
    """
    Loads data from an LMDB file and returns it as a list of (key, value) pairs.
    """
    env = lmdb.open(lmdb_path, readonly=True)
    data = []
    with env.begin() as txn:
        cursor = txn.cursor()
        for key, value in cursor:
            data.append((key, value))

    return data

def save_lmdb_data(lmdb_path_img, lmdb_path_label, img_data, label_data):
    """
    Saves images and labels to LMDB, no decoding and reencoding.
    """
    env_imgs = lmdb.open(lmdb_path_img, map_size=MAP_SIZE_IMG)
    env_labels = lmdb.open(lmdb_path_label, map_size=MAP_SIZE_META)
        
    with (
        env_imgs.begin(write=True) as txn_imgs,
        env_labels.begin(write=True) as txn_labels,
    ):
        # Iterate through img_data and label_data, assuming each contains (key, data)
        for (img_key, img_encoded), (label_key, label) in tqdm(zip(img_data, label_data), total=len(img_data)):
            # Ensure keys match (img_key should be the same in both img_data and label_data)
            if img_key != label_key:
                print(f"Warning: Mismatched keys! img_key: {img_key}, label_key: {label_key}")
                continue  # Skip if keys don't match

            txn_imgs.put(img_key, img_encoded)  
            txn_labels.put(label_key, label)
 
    env_imgs.close()
    env_labels.close()




def load_all_datasets(main_folder):
    """
    Load all datasets in the given folder and return combined image and label data.
    """
    img_data = []
    label_data = []
    
    for dataset in os.listdir(main_folder):
        print(f"Loading dataset: {dataset}")
        dataset_path = os.path.join(main_folder, dataset)
        print(dataset_path)
        if dataset_path.endswith("_imgs"):
            img_data.extend(load_lmdb_data(dataset_path))
        elif dataset_path.endswith("_labels"):
            label_data.extend(load_lmdb_data(dataset_path))
        else:
            print(f"Skipping {dataset_path}")
            print("we are fucked if this contains a valid path")
                
    return img_data, label_data

def split_and_save_data(main_folder, output_folder, test_size=0.2):
    """
    Loads all datasets, splits the data into train and test, and saves them in the output folder.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Load all datasets
    img_data, label_data = load_all_datasets(main_folder)
    
    print(f"Total data loaded: {len(img_data)} images and {len(label_data)} labels.")

    # Split dataset into train and test
    train_imgs, test_imgs = train_test_split(img_data, test_size=test_size,shuffle=True, random_state=43)
    train_labels, test_labels = train_test_split(label_data, test_size=test_size,shuffle=True, random_state=43)

    # Save the split data to LMDB
    save_lmdb_data(os.path.join(output_folder, "TRAIN_imgs"), os.path.join(output_folder, "TRAIN_labels"), train_imgs, train_labels)
    save_lmdb_data(os.path.join(output_folder, "TEST_imgs"), os.path.join(output_folder, "TEST_labels"), test_imgs, test_labels)

    print(f"Finished processing and saving datasets to {output_folder}")

split_and_save_data(main_folder="/home/nick/Documents/ws24/lmdb", output_folder="/home/nick/Documents/ws24/out", test_size=0.2)


Available memory: 23355.890625 MB
Available memory: 23355.890625 MB
Loading dataset: ZooScan_imgs
/home/nick/Documents/ws24/lmdb/ZooScan_imgs
Loading dataset: ZooScan_labels
/home/nick/Documents/ws24/lmdb/ZooScan_labels
Total data loaded: 13212 images and 13212 labels.


100%|██████████| 2643/2643 [00:00<00:00, 140572.48it/s]


Finished processing and saving datasets to /home/nick/Documents/ws24/out


### Verify balanced train test split

In [ ]:
import os
import lmdb
import imageio.v3 as iio
import numpy as np
from collections import Counter

os.makedirs("./debug_output", exist_ok=True)

def count_entries(lmdb_path):
    env = lmdb.open(lmdb_path, readonly=True)
    
    with env.begin() as txn:
        cursor = txn.cursor()
        count = 0
        for key, _ in cursor:
            count += 1
    env.close()
    
    return count

def load_and_verify_lmdb(lmdb_imgs_path, lmdb_labels_path, debug_output_dir, max_images=10):

    num_images = count_entries(lmdb_imgs_path)
    num_labels = count_entries(lmdb_labels_path)
    
    print(f"Number of images in LMDB: {num_images}")
    print(f"Number of labels in LMDB: {num_labels}")

    env_imgs = lmdb.open(lmdb_imgs_path, readonly=True)
    env_labels = lmdb.open(lmdb_labels_path, readonly=True)

    label_counter = Counter()  # Counter to track label occurrences
    
    with env_imgs.begin() as txn_imgs, env_labels.begin() as txn_labels:
        cursor_imgs = txn_imgs.cursor()
        cursor_labels = txn_labels.cursor()        
        count = 0
        for (img_key, img_value), (label_key, label_value) in zip(cursor_imgs, cursor_labels):
            
            label_decoded = label_value.decode("utf-8")
            
            label_counter[label_decoded] += 1
            
    env_imgs.close()
    env_labels.close()
    
    print(f"Number of unique classes: {len(label_counter)}")
    #for label, count in label_counter.items():
    #    print(f"{label}: {count} occurrences")
    

print("Verification for TEST dataset:")
load_and_verify_lmdb("/home/nick/Documents/ws24/out/TEST_imgs" , "/home/nick/Documents/ws24/out/TEST_labels" ,"./debug_output")
print("\nVerification for TRAIN dataset:")
load_and_verify_lmdb("/home/nick/Documents/ws24/out/TRAIN_imgs" , "/home/nick/Documents/ws24/out/TRAIN_labels" ,"./debug_output")


Verification for TEST dataset:
Number of images in LMDB: 2643
Number of labels in LMDB: 2643
Number of unique classes: 49

Verification for TRAIN dataset:
Number of images in LMDB: 10569
Number of labels in LMDB: 10569
Number of unique classes: 66
